In [21]:
from selenium import webdriver
from datetime import datetime
from time import sleep
import configparser
import os
import sys
#import re
#import pandas as pd
import locale
#driver_path_chrome = r'C:\Users\jonat\chromedriver.exe'
#driver_path_edge = r'C:\Users\jonat\msedgedriver.exe'
locale.setlocale(locale.LC_ALL, "ko_KR.UTF-8")


'ko_KR.UTF-8'

In [22]:
config_file_name = 'price_monitoring.config'

#IMPORT CONFIG FROM FILE .CONFIG
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
driver_path_chrome = cfg['general_config']['driver_path_chrome']
driver_path_edge = cfg['general_config']['driver_path_edge']


In [23]:
def get_price():
    xpath_price = '/html/body/div[2]/section/div[1]/div/div[3]/div[5]/div[1]/div/div[2]/span[1]/strong'
    price_content = driver.find_element_by_xpath(xpath_price).text
    try:
        price_content = price_content.replace('원', '')
    except:
        price_content = price_content
    price = locale.atof(price_content)
    return price

In [24]:
def get_image():
    xpath_image = '/html/body/div[2]/section/div[1]/div/div[1]/div[1]/img'
    image_content = driver.find_element_by_xpath(xpath_image).get_attribute('src')
    return image_content

In [25]:
url_list = [
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='
]
price_list = []
for url in url_list:
    driver = webdriver.Edge(driver_path_edge)
    driver.get(url)
    act_dict = {
        #'title' : get_title(),
        'price' : get_price(),
        'time_utc_now' : datetime.utcnow().strftime("%Y%m%d%H%M%S"),
        #'price_span0' : get_price_span0(),
        #'price_span2' : get_price_span2(),
        #'price_box' : get_price_box(),
        'image_link' : get_image(),
        'link' : url
        }
    price_list.append(act_dict)
    driver.quit()
    sleep(2)
price_list

[{'price': 69900.0,
  'time_utc_now': '20220709072625',
  'image_link': 'https://thumbnail7.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/d6e2/022a1986626056c183a71149d2306a170c7f6b98e56ea7c791a1b4b56405.jpg',
  'link': 'https://www.coupang.com/vp/products/6366671555?isAddedCart='},
 {'price': 296700.0,
  'time_utc_now': '20220709072635',
  'image_link': 'https://thumbnail6.coupangcdn.com/thumbnails/remote/492x492ex/image/rs_quotation_api/if3zrexz/ce09abc068c243e39f3c71ebcdd03bb8.jpg',
  'link': 'https://www.coupang.com/vp/products/5056493492?isAddedCart='},
 {'price': 399000.0,
  'time_utc_now': '20220709072647',
  'image_link': 'https://thumbnail10.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/8210/3280c89ac6a383324216a0fbf8b7f56dbf65b27d38a0b64bcf99b41f8549.jpg',
  'link': 'https://www.coupang.com/vp/products/4978269600?isAddedCart='}]

In [26]:
type(price_list[0]['price'])

float

In [27]:
#driver = webdriver.Edge(driver_path_edge)
#driver.get(r'https://www.coupang.com/vp/products/6366671555?isAddedCart=')
#xpath_image = "/html/body/div[2]/section/div[1]/div/div[1]/div[1]/img"
#image_content = driver.find_element_by_xpath(xpath_image).get_attribute('src')
#image_content

In [28]:
datetime.utcnow().strftime("%Y%m%d%H%M%S")

'20220709072651'